In [2]:
from aut import *
from pyspark.sql.functions import asc, countDistinct, first

In [3]:
images = WebArchive(sc, sqlContext, "/home/nruest/Projects/au/sample-data/geocities") \
  .images() \
  .select(remove_prefix_www(extract_domain("url")).alias("domain"), "url", "md5")

In [4]:
images.show(5, True)

[2022-05-28T00:11:17.610Z - 00000 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143512-00103-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


22/05/27 20:11:18 WARN PDFParser: J2KImageReader not loaded. JPEG2000 files will not be processed.
See https://pdfbox.apache.org/2.0/dependencies.html#jai-image-io
for optional dependencies.

22/05/27 20:11:18 WARN TesseractOCRParser: Tesseract OCR is installed and will be automatically applied to image files unless
you've excluded the TesseractOCRParser from the default parser.
Tesseract may dramatically slow down content extraction (TIKA-2359).
As of Tika 1.15 (and prior versions), Tesseract is automatically called.
In future versions of Tika, users may need to turn the TesseractOCRParser on via TikaConfig.
22/05/27 20:11:18 WARN SQLite3Parser: org.xerial's sqlite-jdbc is not loaded.
Please provide the jar on your classpath to parse sqlite files.
See tika-parsers/pom.xml for the correct version.


+-------------+--------------------+--------------------+
|       domain|                 url|                 md5|
+-------------+--------------------+--------------------+
|geocities.com|http://geocities....|17827882f7bf42860...|
|geocities.com|http://geocities....|899bc6e3309b0fc78...|
|geocities.com|http://geocities....|955f6c342ffed6823...|
|geocities.com|http://geocities....|47718718ddfd7d43a...|
|geocities.com|http://geocities....|8ada65828daff258e...|
+-------------+--------------------+--------------------+
only showing top 5 rows



In [5]:
links = images.groupBy("md5") \
              .count() \
              .where(countDistinct("domain")>=2)

In [6]:
result = images.join(links, "md5") \
               .groupBy("domain", "md5") \
               .agg(first("url").alias("image_url")) \
               .orderBy(asc("md5"))

In [7]:
result.show(10, False)

[2022-05-28T00:11:22.907Z - 00012 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143243-00104-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-28T00:11:22.907Z - 00011 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143512-00103-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


[2022-05-28T00:11:23.366Z - 00001 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143512-00103-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-28T00:11:23.412Z - 00008 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143451-00102-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-28T00:11:23.412Z - 00003 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-28T00:11:23.412Z - 00006 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143340-00105-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-28T00:11:23

[2022-05-28T00:20:43.093Z - 00013 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


[2022-05-28T00:21:25.142Z - 00014 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143351-00117-ia400103.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


[2022-05-28T00:21:36.821Z - 00015 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142649-00105-ia400111.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


[2022-05-28T00:21:40.555Z - 00016 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143340-00105-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-28T00:21:54.640Z - 00017 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143841-00136-ia400104.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


[2022-05-28T00:22:03.398Z - 00018 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143451-00102-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-28T00:22:05.184Z - 00019 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142731-00177-ia400130.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


[2022-05-28T00:22:09.652Z - 00020 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143300-00114-ia400112.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


+--------------------------+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|domain                    |md5                             |image_url                                                                                                                                                     |
+--------------------------+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|stevenspointjournal.com   |022ac0cde4560864fc99a3d9e5210f6b|http://cmsimg.stevenspointjournal.com/apps/pbcsi.dll/bilde?Avis=U0&Dato=20091026&Kategori=CWS01&Lopenr=910260803&Ref=PH&NewTbl=1&Itemnr=1&maxW=130&Border=0   |
|marshfieldnewsherald.com  |022ac0cde4560864fc99a3d9e5210f6b|http://cmsimg.marshfieldnewsherald.com/apps/pbcsi.dll/b